##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classificação de texto com o TensorFlow Lite Model Maker

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Veja em TensorFlow.org</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte em GitHub</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/models/modify/model_maker/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

A [biblioteca TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) (criador de modelos do TF Lite) simplifica o processo de adaptar e converter um modelo do TensorFlow para dados de entrada específicos ao implantar esse modelo em aplicativos de aprendizado de máquina em dispositivos.

Este notebook apresenta um exemplo completo que utiliza a biblioteca Model Maker para ilustrar a adaptação e conversão de um modelo de classificação de texto usado com frequência para classificar avaliações de filmes em um dispositivo móvel. O modelo classifica textos em categorias predefinidas. As entradas devem ser texto pré-processado, e as saídas são as probabilidades das categorias. O dataset usado neste tutorial é composto por avaliações de filmes positivas e negativas.

## Pré-requisitos


### Instale os pacotes obrigatórios

Para executar este exemplo, instale os pacotes exigidos, incluindo o pacote do Model Maker no [repositório do GitHub](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker
!pip uninstall tflite_support_nightly
!pip install tflite_support_nightly

Importe os pacotes necessários.

In [ ]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

from tflite_support.task import core
from tflite_support.task import processor
from tflite_support.task import text

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

### Baixe os dados de treinamento de amostra

Neste tutorial, usaremos o [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank), uma das tarefas no referencial [GLUE](https://gluebenchmark.com/). Ele contém 67.349 avaliações de filme para treinamento e 872 avaliações de filmes para teste. O dataset tem duas classes: avaliações de filmes positivas e negativas.

In [ ]:
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://dl.fbaipublicfiles.com/glue/data/SST-2.zip',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

O dataset SST-2 é armazenado no formato TSV. A única diferença entre TSV e CSV é que o TSV usa o caractere de tabulação `\t` como delimitador em vez da vírgula `,` usada no formato CSV.

Veja abaixo as 5 primeiras linhas do dataset de treinamento. label=0 significa negativo, e label=1 significa positivo.

Frase | label |  |  |
--- | --- | --- | --- | ---
hide new secretions from the parental units (oculta novas secreções das unidades parentais) | 0 |  |  |
contains no wit , only labored gags (não contém sabedoria, só piadas cansativas) | 0 |  |  |
that loves its characters and communicates something rather beautiful about human nature (que ama seus personagens e comunica algo lindo sobre a natureza humana) | 1 |  |  |
remains utterly satisfied to remain the same throughout (permanece totalmente satisfeito para permanecer igual o filme todo) | 0 |  |  |
on the worst revenge-of-the-nerds clichés the filmmakers could dredge up (nos piores clichês de vingança dos nerds que os cineastas puderam criar) | 0 |  |  |

Agora, vamos carregar o dataset em um dataframe do Pandas e alterar os nomes de rótulos atuais (`0` e `1`) para rótulos mais legíveis por humanos (`negative` – negativa e `positive` – positiva) e usá-los para o treinamento do modelo.


In [ ]:
import pandas as pd

def replace_label(original_file, new_file):
  # Load the original file to pandas. We need to specify the separator as
  # '\t' as the training data is stored in TSV format
  df = pd.read_csv(original_file, sep='\t')

  # Define how we want to change the label name
  label_map = {0: 'negative', 1: 'positive'}

  # Excute the label change
  df.replace({'label': label_map}, inplace=True)

  # Write the updated dataset to a new file
  df.to_csv(new_file)

# Replace the label name for both the training and test dataset. Then write the
# updated CSV dataset to the current folder.
replace_label(os.path.join(os.path.join(data_dir, 'train.tsv')), 'train.csv')
replace_label(os.path.join(os.path.join(data_dir, 'dev.tsv')), 'dev.csv')

## Início rápido

Existem cinco etapas para treinar um modelo de classificação de texto:

**Etapa 1 – Escolha uma arquitetura de modelo de classificação de texto**

Usaremos a arquitetura de modelo de média de embedding de palavra, que vai gerar um modelo pequeno e rápido, com uma exatidão razoável.

In [ ]:
spec = model_spec.get('average_word_vec')

O Model Maker também oferece suporte a outras arquiteturas de modelo, como [BERT](https://arxiv.org/abs/1810.04805). Se você tiver interesse em aprender outras arquiteturas, confira a seção [Escolha uma arquitetura de modelo para classificador de texto](#scrollTo=kJ_B8fMDOhMR) abaixo.

**Etapa 2 – Carregue os dados de treinamento e teste, depois pré-processe-os de acordo com um `model_spec` específico.**

O Model Maker recebe dados de entrada no formato CSV. Vamos carregar o dataset de treinamento e teste com o nome de rótulos legível por humanos que criamos anteriormente.

Cada arquitetura de modelo requer que os dados de entrada sejam processados de uma forma específica. `DataLoader` lê os requisitos em `model_spec` e executa o pré-processamento necessário automaticamente.

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      is_training=False)

**Etapa 3 – Treine o modelo do TensorFlow com os dados de treinamento**

O modelo de média de embedding de palavra usa `batch_size = 32` por padrão. Portanto, você verá que ele leva 2.104 passos para percorrer as 67.349 frases do dataset de treinamento. Vamos treinar o modelo com 10 épocas, ou seja, percorrer todo o dataset de treinamento 10 vezes.

In [ ]:
model = text_classifier.create(train_data, model_spec=spec, epochs=10)

**Etapa 4 – Avalie o modelo com os dados de teste**

Após treinar o modelo de classificação de texto usando as frases do dataset de treinamento, usaremos as 872 frases restantes no dataset de teste para avaliar o desempenho do modelo com os novos dados que ele nunca viu.

Como o tamanho padrão do lote é 32, vai demorar 28 passos para percorrer as 872 frases do dataset de teste.

In [ ]:
loss, acc = model.evaluate(test_data)

**Etapa 5 – Exporte para um modelo do TensorFlow Lite**

Vamos exportar o classificador de texto que treinamos para o formato do TensorFlow Lite. Especificaremos para qual pasta o modelo deverá ser exportado. Por padrão, o modelo float do TF Lite é exportado para a arquitetura de modelo de média de embedding de palavra.

In [ ]:
model.export(export_dir='average_word_vec')

Você pode baixar o arquivo do modelo do TensorFlow Lite pela barra lateral esquerda do Colab. Acesse a pasta `average_word_vec` que especificamos no parâmetro `export_dir` acima, clique com o botão direito no arquivo `model.tflite` e selecione `Download` (Baixar) para baixá-lo para seu computador local.

Este modelo pode ser integrado a um aplicativo para Android ou iOS usando a [API NLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/nl_classifier) da [biblioteca Task do TensorFlow Lite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

Confira mais detalhes de como o modelo é usado em um aplicativo funcional no [exemplo de aplicativo para classificação de texto do TF Lite](https://github.com/tensorflow/examples/blob/master/lite/examples/text_classification/android/lib_task_api/src/main/java/org/tensorflow/lite/examples/textclassification/client/TextClassificationClient.java#L54).

*Observação: o Android Studio Model Binding ainda não oferece suporte à classificação de texto, então use a TensorFlow Lite Task Library.*

*Observação 2: há um arquivo `model.json` na mesma pasta que do modelo do TF Lite. Ele contém a representação em JSON dos [metadados](https://www.tensorflow.org/lite/models/convert/metadata) empacotados dentro do modelo do TensorFlow Lite. Os metadados do modelo ajudam a TF Lite Task Library a saber o que o modelo faz e como pré/pós-processar os dados para o modelo. Você não precisa baixar o arquivo `model.json`, pois ele tem apenas fins informativos, e seu conteúdo já está dentro do arquivo do TF Lite.*

*Observação 3: se você treinar um modelo de classificação de texto usando a arquitetura MobileBERT ou BERT-Base, precisará usar a [API BertNLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) para integrar o modelo treinado a um aplicativo para dispositivos móveis.*

As próximas seções explicam o exemplo passo a passo para mostrar mais detalhes.

**Etapa 6 – Use a `TFLite Task Library` para demonstrar como utilizar os modelos treinados**

Leia o arquivo dev.csv nos dados de frase para fazer previsões com o modelo treinado.

In [ ]:
sentence_data = pd.read_csv('/content/dev.csv', index_col=0)
sentence_data

Parâmetro de configuração do modelo:

In [ ]:
# Name of the TFLite text classification model.
_MODEL = '/content/average_word_vec/model.tflite'
# Whether to run the model on EdgeTPU.
_ENABLE_EDGETPU = False
# Number of CPU threads to run the model.
_NUM_THREADS = 4

Inicialize o modelo

Também podemos alterar parâmetros como `file_name`, `use_coral` e `num_threads`, o que pode afetar os resultados do modelo. Veja quais parâmetros podem ser ajustados.

- `file_name`: nome do modelo de classificação de texto do TF Lite.
- `use_coral`: se for true (verdadeiro), a inferência será delegada a um dispositivo Coral Edge TPU conectado.
- `num_threads`: número de threads de CPU que executarão o modelo.

In [ ]:
# Initialize the text classification model.
base_options = core.BaseOptions(file_name=_MODEL, use_coral=_ENABLE_EDGETPU, num_threads=_NUM_THREADS)
options = text.NLClassifierOptions(base_options)

# Create NLClassifier from options.
classifier = text.NLClassifier.create_from_options(options)

Faça a previsão usando a `TF Lite Task Library`.

In [ ]:
for idx in range(20):
  sentence = sentence_data['sentence'].iloc[idx]
  label = sentence_data['label'].iloc[idx]
  text_classification_result = classifier.classify(sentence)
  classification_list = text_classification_result.classifications[0].categories

  # Sort output by probability descending.
  predict_label = sorted(
      classification_list, key=lambda item: item.score, reverse=True)[0]

  print('truth_label: {} -----> predict_label: {}'.format(label, predict_label.category_name))

## Escolha uma arquitetura de modelo para classificador de texto

Cada objeto `model_spec` representa um modelo específico de classificador de texto.. Atualmente, o TensorFlow Lite Model Maker tem suporte a modelos [MobileBERT](https://arxiv.org/pdf/2004.02984.pdf), média de embedding de palavra e modelos [BERT-Base](https://arxiv.org/pdf/1810.04805.pdf).

Modelo com suporte | Nome de model_spec | Descrição do modelo | Tamanho do modelo
--- | --- | --- | ---
Média de embedding de palavra | 'average_word_vec' | Média de embedding de palavra de texto com ativação RELU. | &lt; 1 MB
<a>MobileBERT</a> | 'mobilebert_classifier' | 4,3 vezes menor e 5,5 vezes mais rápido do que o BERT-Base, alcançando resultados competitivos, adequados para aplicativos em dispositivos. | 25 MB com quantização <br> 100 MB sem quantização
<a>BERT-Base</a> | 'bert_classifier' | Modelo BERT padrão amplamente usado em tarefas de NLP. | 300 MB

No início rápido, usamos o modelo de média de embedding de palavras. Vamos alterar para [MobileBERT](https://arxiv.org/pdf/2004.02984.pdf) para treinar um modelo com exatidão maior.

In [ ]:
mb_spec = model_spec.get('mobilebert_classifier')

## Carregue os dados de treinamento

Você pode carregar seu próprio dataset neste tutorial. Para carregá-lo, use a barra lateral esquerda no Colab.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_text_classification.png" width="800" hspace="100" alt="Subir arquivo">

Se você preferir não carregar o dataset na nuvem, pode executar a biblioteca localmente de acordo com este [guia](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

Para simplificar, vamos reutilizar o dataset SST-2 baixado anteriormente. Vamos usar o método `DataLoader.from_csv` para carregar os dados.

Observação: como alteramos a arquitetura do modelo, vamos precisar recarregar o dataset de treinamento e de teste para aplicar a nova lógica de pré-processamento.

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=False)

A biblioteca Model Maker também oferece suporte ao método `from_folder()` para carregar dados. Ele pressupõe que os dados de texto da mesma classe estejam no mesmo subdiretório e que o nome da subpasta seja o nome da classe. Cada arquivo de texto contém uma amostra de avaliação de filme. O parâmetro `class_labels` é usado para especificar quais são as subpastas.

## Treine um modelo do TensorFlow

Treine um modelo de classificação de texto usando os dados de treinamento.

*Observação: como o MobileBERT é um modelo complexo, cada época de treinamento demora cerca de 10 minutos em uma GPU do Colab. Você deve usar um runtime de GPU.*

In [ ]:
model = text_classifier.create(train_data, model_spec=mb_spec, epochs=3)

Confira a estrutura detalhada do modelo.

In [ ]:
model.summary()

## Avaliar o modelo

Avalie o modelo que acabamos de treinar usando os dados de teste e mensure os valores de perda e precisão.

In [ ]:
loss, acc = model.evaluate(test_data)

## Exporte para um modelo do TensorFlow Lite

Converta o modelo treinado para o formato de modelos do TensorFlow Lite com [metadados](https://www.tensorflow.org/lite/models/convert/metadata) para poder usá-lo posteriormente em um aplicativo de aprendizado de máquina em dispositivos. O arquivo de rótulos e o arquivo de vocabulário são incorporados aos metadados. O nome de arquivo padrão do TF Lite é `model.tflite`.

Em diversos aplicativos de aprendizado de máquina em dispositivos, o tamanho do modelo é um fator importante. Portanto, recomendamos aplicar quantização no modelo para deixá-lo menor e possivelmente mais rápido. Para modelos BERT e MobileBERT, a técnica padrão de quantização pós-treinamento é a quantização de intervalo dinâmico.

In [ ]:
model.export(export_dir='mobilebert/')

O arquivo do modelo do TensorFlow Lite pode ser integrado a um aplicativo para dispositivos móveis usando a [API BertNLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) na [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview). Observação: isso é **diferente** da API `NLClassifier` usada para integrar um classificador de texto treinado com a arquitetura de modelo de média do vetor de palavras.

Confira abaixo os formatos de exportação permitidos:

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.VOCAB`
- `ExportFormat.SAVED_MODEL`

Por padrão, ele exporta somente o modelo do TensorFlow Lite contendo os metadados do modelo. Você também pode optar por exportar outros arquivos relacionados ao modelo para avaliá-los melhor. Por exemplo, é possível exportar somente o arquivo de rótulos e o arquivo de vocabulário da seguinte forma:

In [ ]:
model.export(export_dir='mobilebert/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

Você pode avaliar o modelo do TF Lite com o método `evaluate_tflite` para mensurar a exatidão. Ao converter o modelo do TensorFlow treinado para o formato do TF Lite e aplicar quantização, a exatidão pode ser afetada, então é recomendável avaliar a exatidão do modelo do TF Lite antes da implantação.

In [ ]:
accuracy = model.evaluate_tflite('mobilebert/model.tflite', test_data)
print('TFLite model accuracy: ', accuracy)

## Uso avançado

A função `create` é a função que a biblioteca Model Maker usa para criar modelos. O parâmetro `model_spec` define a especificação do modelo. No momento, há suporte às classes `AverageWordVecSpec` e `BertClassifierSpec`. A função `create` consiste nas seguintes etapas:

1. Cria o modelo de classificação de texto de acordo com `model_spec`.
2. Treina o modelo de classificador. As épocas padrão e o tamanho de lote padrão são definidos pelas duas variáveis `default_training_epochs` e `default_batch_size` no objeto `model_spec`.

Esta seção abrange tópicos de uso avançado, como ajustar o modelo e os hiperparâmetros de treinamento.

### Personalize os hiperparâmetros do modelo MobileBERT

Os parâmetros do modelo que podem ser ajustados são:

- `seq_len`: tamanho da sequência a ser alimentada no modelo.
- `initializer_range`: desvio padrão do `truncated_normal_initializer` para inicializar as matrizes de pesos.
- `trainable`: booleano que especifica se a camada pré-treinada é treinável.

Os parâmetros do pipeline de treinamento que podem ser ajustados são:

- `model_dir`: local dos arquivos de checkpoint do modelo. Caso não seja definido, será usado um diretório temporário.
- `dropout_rate`: taxa de dropout.
- `learning_rate`: taxa de aprendizado inicial para o otimizador Adam.
- `tpu`: endereço da TPU à qual se conectar.

Por exemplo: você pode definir `seq_len=256` (o padrão é 128), o que permite ao modelo classificar textos maiores.

In [ ]:
new_model_spec = model_spec.get('mobilebert_classifier')
new_model_spec.seq_len = 256

### Personalize os hiperparâmetros do modelo de média de embedding de palavra

Você pode ajustar a infraestrutura do modelo, como as variáveis `wordvec_dim` e `seq_len` na classe  `AverageWordVecSpec`.


Por exemplo: você pode treinar o modelo com um valor maior de `wordvec_dim`. Observação: você precisa construir um novo `model_spec` se modificar o modelo.

In [ ]:
new_model_spec = AverageWordVecSpec(wordvec_dim=32)

Obtenha os dados pré-processados.

In [ ]:
new_train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=new_model_spec,
      is_training=True)

Treine o novo modelo.

In [ ]:
model = text_classifier.create(new_train_data, model_spec=new_model_spec)

### Ajuste os hiperparâmetros de treinamento

Você também pode ajustar os hiperparâmetros de treinamento, como `epochs` e `batch_size`, o que impacta a exatidão do modelo. Por exemplo:

- `epochs`: mais épocas podem levar a uma exatidão melhor, mas podem causar overfitting.
- `batch_size`: número de amostras a serem usadas em um passo de treinamento.

Por exemplo: você pode treinar com mais épocas.

In [ ]:
model = text_classifier.create(new_train_data, model_spec=new_model_spec, epochs=20)

Avalie o modelo retreinado recentemente com 20 épocas de treinamento

In [ ]:
new_test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=new_model_spec,
      is_training=False)

loss, accuracy = model.evaluate(new_test_data)

### Altere a arquitetura do modelo

É possível alterar o modelo mudando `model_spec`. O exemplo abaixo mostra como alterar para o modelo BERT-Base.

Altere `model_spec` para o modelo BERT-Base para o classificador de texto.

In [ ]:
spec = model_spec.get('bert_classifier')

As outras etapas são as mesmas.

### Personalize a quantização pós-treinamento em um modelo do TensorFlow Lite

A [quantização pós-treinamento](https://www.tensorflow.org/lite/performance/post_training_quantization) é uma técnica de conversão que pode reduzir o tamanho do modelo e a latência de inferência, além de aumentar a velocidade de inferência da CPU e do acelerador de hardware com uma pequena redução da exatidão do modelo. A quantização é amplamente utilizada para otimizar o modelo.

A biblioteca Model Maker aplica uma técnica padrão de quantização pós-treinamento ao exportar o modelo. Se você quiser personalizar a quantização pós-treinamento, o Model Maker oferece suporte a diversas opções usando [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig). Vejamos a quantização de float 16 como exemplo. Primeiro, definimos a configuração de quantização.

```python
config = QuantizationConfig.for_float16()
```

Em seguida, exportamos o modelo do TensorFlow Lite com essa configuração.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Saiba mais

Leia o exemplo de [classificação de texto](https://www.tensorflow.org/lite/examples/text_classification/overview) para aprender os detalhes técnicos. Confira mais informações em:

- [Guia](https://www.tensorflow.org/lite/models/modify/model_maker) e [referência da API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) do TensorFlow Lite Model Maker.
- Task Library: [NLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/nl_classifier) e [BertNLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) para implantação.
- Aplicativos de referência completos para [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/text_classification/android) e [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/text_classification/ios).